In [1]:
import numpy as np

In [25]:
class Hebb(object):
    def __init__(self):
        self.weights = np.zeros((30,30))
        self.n = 30
    
    def f(self,x):
        x[x>0] = 1
        x[x<=0] = -1
        return x
    
    def wild_hebb(self, vec):
        raw = self.weights * vec.repeat(self.n).reshape(self.n, self.n)
        new = np.zeros_like(raw.reshape(30, 30))
        u, v = new.shape
        for i in range(u):
            for j in range(v):
                new[i, j] = np.sum(raw[0:j+1, i])
        delta_w = vec.repeat(self.n).reshape(self.n, self.n) * new
        return delta_w

    def train_new(self, input_vector, iter_, rate):
        for i in range(iter_):
            for vec in input_vector:
                vec = np.matrix(vec)
		#这里可以使用广义hebb算法中的权重更新算法
                wild_hebb = self.wild_hebb(vec)
                delta = np.ones_like(wild_hebb)
                index = np.where(wild_hebb < 0)
                delta[index] = -1
                self.weights = self.weights + rate * vec.getT().dot(vec)
            
    def train(self, input_vector, iter_, rate):
        for i in range(iter_):
            for vec in input_vector:
                vec = np.matrix(vec)
		#这里可以使用广义hebb算法中的权重更新算法
                self.weights = self.weights + rate * vec.getT().dot(vec)
        return self.weights
    
    def predict(self, input_vector):
         return self.weights.dot(np.matrix(input_vector).getT())

In [70]:
def add_noise(x,threshold):
    if len(x.shape) >=2:
        for i in range(x.shape[1]):
            random_c = np.random.randint(0,1000)
            if random_c < threshold*1000:
                x[0,i] = -x[0,i]
    else:
        print(x.shape)
        for i in range(len(x)):
            random_c = np.random.randint(0,1000)
            if random_c < threshold*1000:
                x[i] = -x[i]
            
    return x

In [71]:
zero = [
    -1, 1, 1, 1, -1,
    1, -1, -1, -1, 1,
    1, -1, -1, -1, 1,
    1, -1, -1, -1, 1,
    1, -1, -1, -1, 1,
    -1, 1, 1, 1, -1
    ]

one = [
    -1, 1, 1, -1, -1,
    -1, -1, 1, -1, -1,
    -1, -1, 1, -1, -1,
    -1, -1, 1, -1, -1,
    -1, -1, 1, -1, -1,
    -1, -1, 1, -1, -1
    ]

two = [
    1, -1, 1, -1, -1,
    -1, -1, -1, 1, -1,
    -1, -1, -1, 1, -1,
    -1, 1, 1, -1, -1,
    1, -1, -1, -1, -1,
    1, 1, 1, 1, 1,
    ]
half_zero = [
1, 1, 1, 1, 1,
1, -1, -1, -1, 1,
1, -1, 1, -1, 1,
1, -1, -1, -1, 1,
-1, -1, -1, -1, -1,
-1, -1, -1, -1, -1,
]


### 原始

In [72]:
hebb  = Hebb()
w=hebb.train([zero, one, two], 600, 0.2)
pre = hebb.predict(half_zero)
print(pre.reshape((6,5)))
def draw_bin_image(image_matrix):
    for row in image_matrix.tolist():
        print('| ' + ' '.join(' *'[int(val)] for val in row))


[[-3.36000000e+03  3.36000000e+03  9.60000000e+02  2.40000000e+03
  -9.60000000e+02]
 [ 2.40000000e+03 -9.60000000e+02 -3.06954462e-11 -3.36000000e+03
   2.40000000e+03]
 [ 2.40000000e+03 -9.60000000e+02 -3.06954462e-11 -3.36000000e+03
   2.40000000e+03]
 [ 2.40000000e+03 -3.36000000e+03 -2.40000000e+03 -9.60000000e+02
   2.40000000e+03]
 [ 3.06954462e-11 -9.60000000e+02 -3.06954462e-11 -9.60000000e+02
   2.40000000e+03]
 [-3.36000000e+03  3.06954462e-11  9.60000000e+02  3.06954462e-11
  -3.36000000e+03]]


In [73]:
def acti_fun(x):
    for i in range(x.size):
        if x[i] > 0:
            x[i] =1
        else:
            x[i] =0
    return x


In [74]:
draw_bin_image(acti_fun(np.array(half_zero)).reshape((6,5)))

| * * * * *
| *       *
| *   *   *
| *       *
|          
|          


In [75]:
draw_bin_image(acti_fun(pre).reshape((6,5)))

|   * * *  
| *       *
| *       *
| *       *
| *       *
|   * * *  


In [77]:
zero_noise = add_noise(np.matrix(zero),0.2)
draw_bin_image(acti_fun(np.array(zero_noise).reshape(30,)).reshape((6,5)))

|   *   *  
| *       *
|         *
| * *     *
| *     * *
|   * * *  


In [78]:
pre = hebb.predict(zero_noise)
draw_bin_image(acti_fun(pre).reshape((6,5)))

|   * * *  
| *       *
| *       *
| *       *
| *       *
|   * * *  


### 广义

In [81]:
hebb  = Hebb()
## here changed
w=hebb.train_new([zero, one, two], 600, 0.2)
pre = hebb.predict(half_zero)
print(pre.reshape((6,5)))
def draw_bin_image(image_matrix):
    for row in image_matrix.tolist():
        print('| ' + ' '.join(' *'[int(val)] for val in row))

[[-3.36000000e+03  3.36000000e+03  9.60000000e+02  2.40000000e+03
  -9.60000000e+02]
 [ 2.40000000e+03 -9.60000000e+02 -3.06954462e-11 -3.36000000e+03
   2.40000000e+03]
 [ 2.40000000e+03 -9.60000000e+02 -3.06954462e-11 -3.36000000e+03
   2.40000000e+03]
 [ 2.40000000e+03 -3.36000000e+03 -2.40000000e+03 -9.60000000e+02
   2.40000000e+03]
 [ 3.06954462e-11 -9.60000000e+02 -3.06954462e-11 -9.60000000e+02
   2.40000000e+03]
 [-3.36000000e+03  3.06954462e-11  9.60000000e+02  3.06954462e-11
  -3.36000000e+03]]


In [82]:
draw_bin_image(acti_fun(np.array(half_zero)).reshape((6,5)))

| * * * * *
| *       *
| *   *   *
| *       *
|          
|          


In [83]:
pre = hebb.predict(half_zero)
draw_bin_image(acti_fun(pre).reshape((6,5)))

|   * * *  
| *       *
| *       *
| *       *
| *       *
|   * * *  


In [84]:
zero_noise = add_noise(np.matrix(zero),0.2)
draw_bin_image(acti_fun(np.array(zero_noise).reshape(30,)).reshape((6,5)))

| * * *    
| *       *
|     * *  
|       * *
|   *     *
| * * * *  


In [85]:
pre = hebb.predict(zero_noise)
draw_bin_image(acti_fun(pre).reshape((6,5)))

|   * * *  
| *       *
| *       *
| *       *
| *       *
|   * * *  
